**Jeel**, current issues that need working

* some customers make foreign transactions, i already converted those transactions to their home currency.

BUT some people seems to have more than one country listed as thier home. I was thinking of looking at these cases and choosing one country to assign as their home



# Smart Budget Planner
Data Prepared by Raphael Lu and Jeel Faldu

In [1]:
import pandas as pd
import numpy as np
import json

import kagglehub
from kagglehub import KaggleDatasetAdapter

## Introduction

The purpose of this project is to prepare data for a Smart Budget Planner Project. Specifically, we take a kaggle dataset of international customer credit card transactions from 01 October to 30 October 2024


**Final JSON Structure**

1.  country #('UK' 'Brazil' 'Japan' 'Australia' 'Nigeria' 'Germany' 'Mexico' 'Russia' 'France' 'Canada' 'Singapore' 'USA')

  * country_aggregate-data
      * summary_stats
           * mean_total_spend
           * median_total_spend
           * std_total_spend
      
      * currency
      
      * categories #(['Restaurant' 'Entertainment' 'Grocery' 'Gas' 'Healthcare' 'Education' 'Travel' 'Retail'])
           * category (i.e. Resturant)
               * cat_mean
               * cat_median
               * cat_std
               * sub_catagories

  * users
      * CUST_00001
           * total_spending
           * categories
                * category (i.e. Restaurant)
                     * mean
                     * highest_transaction
                          * amount
                          * sub_category
                     * cat_zscore
                     * over_benchmark: bool
                     * spending_by_sub
                        * sub_cat_1 (i.e. 'fast_food')
                * ....

2. country ...



## Data Loading and Pre-processing

### Historic Currency Conversion Rates

During data exploration, it was found that multiple customers made foreign transaction. The dictionary below was obtain with the use of Claude.

**Currency Conversion Rates**  
Source: HM Revenue & Customs (HMRC) - October 2024 Monthly Exchange Rates
Retrieved via: Claude AI (Anthropic) web search on November 9, 2025
Original source: https://www.trade-tariff.service.gov.uk/exchange_rates/view/2024-10
Date Range: October 1-31, 2024

In [2]:
conversion_to_usd = {
    'USA': 1.0,
    'UK': 0.757,        # 1 GBP = 1.32 USD → 1 USD = 0.757 GBP
    'Brazil': 0.182,    # BRL: 7.24 per GBP → 5.48 per USD
    'Japan': 0.00706,   # JPY: 187.4 per GBP → 141.8 per USD
    'Australia': 0.679, # AUD: 1.95 per GBP → 1.47 per USD
    'Nigeria': 0.00061, # NGN: 2165.7 per GBP → 1639 per USD
    'Germany': 0.842,   # EUR: 1.19 per GBP → 1.09 EUR per USD
    'Mexico': 0.0524,   # MXN: 25.21 per GBP → 19.08 per USD
    'Russia': 0.0109,   # RUB: 121.3 per GBP → 91.8 per USD
    'France': 0.842,    # EUR (same as Germany)
    'Canada': 0.736,    # CAD: 1.79 per GBP → 1.36 per USD
    'Singapore': 0.773  # SGD: 1.71 per GBP → 1.29 per USD
}

### df Loading

In [3]:
df = kagglehub.dataset_load(KaggleDatasetAdapter.PANDAS,
                            "ismetsemedov/transactions",
                            "synthetic_fraud_data.csv")

100%|██████████| 2.73G/2.73G [03:02<00:00, 16.1MB/s]


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7483766 entries, 0 to 7483765
Data columns (total 24 columns):
 #   Column               Dtype  
---  ------               -----  
 0   transaction_id       object 
 1   customer_id          object 
 2   card_number          int64  
 3   timestamp            object 
 4   merchant_category    object 
 5   merchant_type        object 
 6   merchant             object 
 7   amount               float64
 8   currency             object 
 9   country              object 
 10  city                 object 
 11  city_size            object 
 12  card_type            object 
 13  card_present         bool   
 14  device               object 
 15  channel              object 
 16  device_fingerprint   object 
 17  ip_address           object 
 18  distance_from_home   int64  
 19  high_risk_merchant   bool   
 20  transaction_hour     int64  
 21  weekend_transaction  bool   
 22  velocity_last_hour   object 
 23  is_fraud             bool   
dty

In [5]:
df.head(5)

,transaction_id,customer_id,card_number,timestamp,merchant_category,merchant_type,merchant,amount,currency,country,...,device,channel,device_fingerprint,ip_address,distance_from_home,high_risk_merchant,transaction_hour,weekend_transaction,velocity_last_hour,is_fraud
0,TX_a0ad2a2a,CUST_72886,6646734767813109,2024-09-30 00:00:01.034820+00:00,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,...,iOS App,mobile,e8e6160445c935fd0001501e4cbac8bc,197.153.60.199,0,False,0,False,"{'num_transactions': 1197, 'total_amount': 334...",False
1,TX_3599c101,CUST_70474,376800864692727,2024-09-30 00:00:01.764464+00:00,Entertainment,gaming,Steam,3368.97,BRL,Brazil,...,Edge,web,a73043a57091e775af37f252b3a32af9,208.123.221.203,1,True,0,False,"{'num_transactions': 509, 'total_amount': 2011...",True
2,TX_a9461c6d,CUST_10715,5251909460951913,2024-09-30 00:00:02.273762+00:00,Grocery,physical,Whole Foods,102582.38,JPY,Japan,...,Firefox,web,218864e94ceaa41577d216b149722261,10.194.159.204,0,False,0,False,"{'num_transactions': 332, 'total_amount': 3916...",False
3,TX_7be21fc4,CUST_16193,376079286931183,2024-09-30 00:00:02.297466+00:00,Gas,major,Exxon,630.60,AUD,Australia,...,iOS App,mobile,70423fa3a1e74d01203cf93b51b9631d,17.230.177.225,0,False,0,False,"{'num_transactions': 764, 'total_amount': 2201...",False
4,TX_150f490b,CUST_87572,6172948052178810,2024-09-30 00:00:02.544063+00:00,Healthcare,medical,Medical Center,724949.27,NGN,Nigeria,...,Chrome,web,9880776c7b6038f2af86bd4e18a1b1a4,136.241.219.151,1,False,0,False,"{'num_transactions': 218, 'total_amount': 4827...",True


In [6]:
print('Unique Categories:', df['merchant_category'].unique())
print()
print('Unique Merchant Types (sub catagories):', df['merchant_type'].unique())
print()
print('Unique Countries:', df['country'].unique())
print()
print('Unique Currencies:', df['currency'].unique())

Unique Categories: ['Restaurant' 'Entertainment' 'Grocery' 'Gas' 'Healthcare' 'Education'
 'Travel' 'Retail']

Unique Merchant Types (sub catagories): ['fast_food' 'gaming' 'physical' 'major' 'medical' 'online' 'hotels'
 'pharmacy' 'premium' 'events' 'supplies' 'airlines' 'local' 'booking'
 'streaming' 'transport' 'casual']

Unique Countries: ['UK' 'Brazil' 'Japan' 'Australia' 'Nigeria' 'Germany' 'Mexico' 'Russia'
 'France' 'Canada' 'Singapore' 'USA']

Unique Currencies: ['GBP' 'BRL' 'JPY' 'AUD' 'NGN' 'EUR' 'MXN' 'RUB' 'CAD' 'SGD' 'USD']


### Drop Irrelevant Columns

In [7]:
# Drop Fraud Cases
df = df[df['is_fraud'] == False]

# Cols to drop
drop_cols = ['card_number', 'city_size', 'card_type', 'card_present',
             'device_fingerprint', 'ip_address',
             'high_risk_merchant','velocity_last_hour','device','is_fraud',
             'transaction_hour','city']

### Create Transactions df

In [8]:
df_transactions = df.copy().drop(drop_cols, axis=1)
df_transactions

,transaction_id,customer_id,timestamp,merchant_category,merchant_type,merchant,amount,currency,country,channel,distance_from_home,weekend_transaction
0,TX_a0ad2a2a,CUST_72886,2024-09-30 00:00:01.034820+00:00,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,mobile,0,False
2,TX_a9461c6d,CUST_10715,2024-09-30 00:00:02.273762+00:00,Grocery,physical,Whole Foods,102582.38,JPY,Japan,web,0,False
3,TX_7be21fc4,CUST_16193,2024-09-30 00:00:02.297466+00:00,Gas,major,Exxon,630.60,AUD,Australia,mobile,0,False
6,TX_e0d7eb37,CUST_89147,2024-09-30 00:00:03.149440+00:00,Grocery,online,Instacart,2606.19,BRL,Brazil,mobile,0,False
7,TX_eb55c2be,CUST_10150,2024-09-30 00:00:06.295911+00:00,Travel,hotels,Westin,828.33,EUR,Germany,web,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
7483761,TX_58a762fa,CUST_11344,2024-10-30 23:59:58.926575+00:00,Retail,online,AliExpress,887.32,SGD,Singapore,web,0,False
7483762,TX_dabcf671,CUST_38961,2024-10-30 23:59:58.950801+00:00,Grocery,physical,Kroger,356.06,EUR,France,web,0,False
7483763,TX_ab567eb4,CUST_75576,2024-10-30 23:59:58.972155+00:00,Grocery,physical,Kroger,391.96,SGD,Singapore,web,0,False
7483764,TX_5ae61846,CUST_82508,2024-10-30 23:59:58.996608+00:00,Retail,online,eBay,601.71,USD,USA,web,0,False


### Timestamp

In [9]:
df_transactions['timestamp'] = pd.to_datetime(df['timestamp'], format='mixed')

In [10]:
df_transactions['timestamp'].min(), df_transactions['timestamp'].max()

(Timestamp('2024-09-30 00:00:01.034820+0000', tz='UTC'),
 Timestamp('2024-10-30 23:59:59.101885+0000', tz='UTC'))

In [11]:
df_transactions['date_transact'] = df_transactions['timestamp'].dt.strftime('%d-%m-%Y')

In [12]:
df_transactions = df_transactions[df_transactions['date_transact'] != 9]
df_transactions.head()

,transaction_id,customer_id,timestamp,merchant_category,merchant_type,merchant,amount,currency,country,channel,distance_from_home,weekend_transaction,date_transact
0,TX_a0ad2a2a,CUST_72886,2024-09-30 00:00:01.034820+00:00,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,mobile,0,False,30-09-2024
2,TX_a9461c6d,CUST_10715,2024-09-30 00:00:02.273762+00:00,Grocery,physical,Whole Foods,102582.38,JPY,Japan,web,0,False,30-09-2024
3,TX_7be21fc4,CUST_16193,2024-09-30 00:00:02.297466+00:00,Gas,major,Exxon,630.60,AUD,Australia,mobile,0,False,30-09-2024
6,TX_e0d7eb37,CUST_89147,2024-09-30 00:00:03.149440+00:00,Grocery,online,Instacart,2606.19,BRL,Brazil,mobile,0,False,30-09-2024
7,TX_eb55c2be,CUST_10150,2024-09-30 00:00:06.295911+00:00,Travel,hotels,Westin,828.33,EUR,Germany,web,0,False,30-09-2024


### Categories and Sub-categories

In [13]:
for cat in df_transactions['merchant_category'].unique():
  mask = df_transactions['merchant_category'] == cat
  print(f'Sub Categories in {cat}', df_transactions[mask]['merchant_type'].unique())
  print()

Sub Categories in Restaurant ['fast_food' 'premium' 'casual']

Sub Categories in Grocery ['physical' 'online']

Sub Categories in Gas ['major' 'local']

Sub Categories in Travel ['hotels' 'airlines' 'booking' 'transport']

Sub Categories in Healthcare ['medical' 'pharmacy']

Sub Categories in Entertainment ['events' 'gaming' 'streaming']

Sub Categories in Education ['supplies' 'online']

Sub Categories in Retail ['physical' 'online']



### Handeling Foreign Transactions

In [14]:
# creates dict (cust_id : homecountry)
home_country_map = df_transactions[df_transactions['distance_from_home'] == 0].groupby('customer_id')['country'].first()

# .map calls on the dict
df_transactions['home_country'] = df_transactions['customer_id'].map(home_country_map)

df_transactions['sales_loc_rate_to_usd'] = df_transactions['country'].map(conversion_to_usd)
df_transactions['home_rate_to_usd'] = df_transactions['home_country'].map(conversion_to_usd)

foreign_mask = df_transactions['distance_from_home'] == 1

df_transactions.loc[foreign_mask, 'amount'] =  (df_transactions.loc[foreign_mask, 'amount'] * df_transactions.loc[foreign_mask, 'sales_loc_rate_to_usd'] / df_transactions.loc[foreign_mask, 'home_rate_to_usd'])

df_transactions.loc[foreign_mask, 'country'] = df_transactions.loc[foreign_mask, 'home_country']

df_transactions = df_transactions.drop(['sales_loc_rate_to_usd', 'home_rate_to_usd', 'home_country'], axis=1)

### Country dfs (dict)

In [15]:
county_df_dict = {}

for country in df_transactions['country'].unique():
  county_df_dict[country] = df_transactions[df_transactions['country'] == country]

In [16]:
print(county_df_dict.keys())

dict_keys(['UK', 'Japan', 'Australia', 'Brazil', 'Germany', 'Nigeria', 'Russia', 'France', 'Canada', 'Mexico', 'Singapore', 'USA'])


In [17]:
home_country_map_2 = df_transactions[df_transactions['distance_from_home'] == 0].groupby('customer_id')['country']
df_multinat = home_country_map_2.nunique()

df_multinat[df_multinat >= 2].max(), df_multinat[df_multinat >= 2]



(2,
 customer_id
 CUST_11396    2
 CUST_11430    2
 CUST_12255    2
 CUST_13670    2
 CUST_14192    2
              ..
 CUST_97591    2
 CUST_98389    2
 CUST_98661    2
 CUST_98694    2
 CUST_99140    2
 Name: country, Length: 118, dtype: int64)

### NOTE TO JEEL: So i left off here. we have 118 customers that have 2 countries listed as home as denoted by the 'distance_from_home' column

------
# JEEL PLEASE WORK BELOW THIS LINE
-----------

In [18]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5989047 entries, 0 to 7483765
Data columns (total 13 columns):
 #   Column               Dtype              
---  ------               -----              
 0   transaction_id       object             
 1   customer_id          object             
 2   timestamp            datetime64[ns, UTC]
 3   merchant_category    object             
 4   merchant_type        object             
 5   merchant             object             
 6   amount               float64            
 7   currency             object             
 8   country              object             
 9   channel              object             
 10  distance_from_home   int64              
 11  weekend_transaction  bool               
 12  date_transact        object             
dtypes: bool(1), datetime64[ns, UTC](1), float64(1), int64(1), object(9)
memory usage: 599.7+ MB


In [19]:
df_transactions.describe()

,amount,distance_from_home
count,5.989047e+06,5.989047e+06
mean,3.883703e+04,1.739484e-01
std,1.469896e+05,3.790651e-01
min,6.720937e+00,0.000000e+00
25%,3.658400e+02,0.000000e+00
50%,8.915900e+02,0.000000e+00
75%,1.427127e+04,0.000000e+00
max,5.114891e+06,1.000000e+00


In [20]:
multi_country_cust = df_multinat[df_multinat >= 2].index

multinat_df = df_transactions[df_transactions['customer_id'].isin(multi_country_cust)]

print("Number of customers with multiple home countries:", len(multi_country_cust))
print("\nDetailed transaction info for these customers:")
display(multinat_df.sort_values('customer_id').head(20))

Number of customers with multiple home countries: 118

Detailed transaction info for these customers:


,transaction_id,customer_id,timestamp,merchant_category,merchant_type,merchant,amount,currency,country,channel,distance_from_home,weekend_transaction,date_transact
2471708,TX_f92cdab6,CUST_11396,2024-10-10 07:17:30.662273+00:00,Travel,transport,Enterprise Rent-A-Car,3639.640000,BRL,Brazil,web,0,False,10-10-2024
5962932,TX_535596db,CUST_11396,2024-10-24 17:18:20.026873+00:00,Gas,local,Highway Gas Stop,538.119103,BRL,Canada,mobile,1,False,24-10-2024
5300835,TX_808a0e26,CUST_11396,2024-10-21 22:54:34.095909+00:00,Restaurant,casual,TGI Fridays,307.050000,CAD,Canada,mobile,0,False,21-10-2024
1625090,TX_fb162a81,CUST_11396,2024-10-06 17:49:41.652553+00:00,Grocery,online,Instacart,11028.680000,BRL,Brazil,web,0,True,06-10-2024
392133,TX_413dd6a1,CUST_11396,2024-10-01 15:42:20.240934+00:00,Grocery,physical,Publix,427.800000,CAD,Canada,mobile,0,False,01-10-2024
2110423,TX_f68e3790,CUST_11396,2024-10-08 18:03:26.675742+00:00,Entertainment,events,LiveNation,9666.530000,BRL,Brazil,web,0,False,08-10-2024
392158,TX_cecdbae9,CUST_11396,2024-10-01 15:42:29.875764+00:00,Restaurant,premium,Nobu,629.442935,USD,Canada,mobile,1,False,01-10-2024
6085922,TX_2b4c82b7,CUST_11396,2024-10-25 06:23:33.780022+00:00,Education,online,edX,774.149484,MXN,Canada,web,1,False,25-10-2024
392480,TX_20bccf7f,CUST_11396,2024-10-01 15:44:31.864421+00:00,Travel,hotels,Hilton,741.487435,JPY,Canada,web,1,False,01-10-2024
5529969,TX_5dcd226f,CUST_11396,2024-10-22 21:33:01.309886+00:00,Entertainment,gaming,Steam,504.520000,CAD,Canada,mobile,0,False,22-10-2024


In [21]:
transactions_per_multinat = (
    multinat_df.groupby('customer_id')
    .size()
    .reset_index(name='num_transactions')
    .sort_values(by='num_transactions', ascending=False)
)

display(transactions_per_multinat)

,customer_id,num_transactions
102,CUST_91730,3214
19,CUST_24836,3014
95,CUST_87928,3005
81,CUST_77434,2983
2,CUST_12255,2960
...,...,...
72,CUST_69207,1805
117,CUST_99140,1763
10,CUST_22424,1708
104,CUST_92533,1696


In [22]:
home_countries = df_multinat.unique()

print("Total number of distinct home countries:", len(home_countries))


Total number of distinct home countries: 2


In [23]:
countries_per_customer = (
    multinat_df.groupby('customer_id')['country']
    .unique()
    .reset_index(name='home_countries')
)

display(countries_per_customer)

,customer_id,home_countries
0,CUST_11396,"[Canada, Brazil]"
1,CUST_11430,"[Russia, Canada]"
2,CUST_12255,"[Nigeria, Russia]"
3,CUST_13670,"[Canada, Brazil]"
4,CUST_14192,"[Brazil, Singapore]"
...,...,...
113,CUST_97591,"[Japan, UK]"
114,CUST_98389,"[Russia, Brazil]"
115,CUST_98661,"[Canada, Japan]"
116,CUST_98694,"[Australia, Brazil]"


In [24]:
home_txns = multinat_df[multinat_df['distance_from_home'] == 0]

txn_count_per_country = (
    home_txns.groupby(['customer_id', 'country'])
    .size()
    .reset_index(name='num_transactions')
)

display(txn_count_per_country)


,customer_id,country,num_transactions
0,CUST_11396,Brazil,755
1,CUST_11396,Canada,851
2,CUST_11430,Canada,923
3,CUST_11430,Russia,838
4,CUST_12255,Nigeria,1215
...,...,...,...
231,CUST_98661,Japan,1237
232,CUST_98694,Australia,1025
233,CUST_98694,Brazil,584
234,CUST_99140,Canada,679


In [25]:
idx = txn_count_per_country.groupby('customer_id')['num_transactions'].idxmax()
dominant_home_country = txn_count_per_country.loc[idx, ['customer_id', 'country']].rename(columns={'country': 'temp_home_country'})

multinat_df = multinat_df.merge(dominant_home_country, on='customer_id', how='left')

display(dominant_home_country)

,customer_id,temp_home_country
1,CUST_11396,Canada
2,CUST_11430,Canada
5,CUST_12255,Russia
6,CUST_13670,Brazil
9,CUST_14192,Singapore
...,...,...
226,CUST_97591,Japan
228,CUST_98389,Brazil
231,CUST_98661,Japan
232,CUST_98694,Australia


In [26]:
multinat_df['sales_loc_rate_to_usd'] = multinat_df['country'].map(conversion_to_usd)
multinat_df['home_rate_to_usd'] = multinat_df['temp_home_country'].map(conversion_to_usd)

foreign_mask = multinat_df['distance_from_home'] == 1
multinat_df.loc[foreign_mask, 'amount'] = (
    multinat_df.loc[foreign_mask, 'amount'] *
    multinat_df.loc[foreign_mask, 'sales_loc_rate_to_usd'] /
    multinat_df.loc[foreign_mask, 'home_rate_to_usd']
)


In [27]:
multinat_df['country'] = multinat_df['temp_home_country']


In [28]:
multinat_df = multinat_df.drop(['temp_home_country', 'sales_loc_rate_to_usd', 'home_rate_to_usd'], axis=1)


In [29]:
df_transactions.update(multinat_df)
display(df_transactions.head())


,transaction_id,customer_id,timestamp,merchant_category,merchant_type,merchant,amount,currency,country,channel,distance_from_home,weekend_transaction,date_transact
0,TX_5595b6f4,CUST_79361,2024-09-30 00:00:21.797955+00:00,Grocery,online,Walmart Grocery,47292.58,JPY,Japan,web,0,False,30-09-2024
2,TX_f8fa6746,CUST_84744,2024-09-30 00:00:56.666078+00:00,Restaurant,casual,TGI Fridays,129.17,USD,Nigeria,mobile,0,False,30-09-2024
3,TX_810559ca,CUST_69207,2024-09-30 00:00:59.893684+00:00,Retail,physical,Home Depot,340.91,EUR,France,web,1,False,30-09-2024
6,TX_bd11459f,CUST_50255,2024-09-30 00:02:21.344056+00:00,Gas,local,Highway Gas Stop,261938.00,NGN,Nigeria,web,0,False,30-09-2024
7,TX_97f03eee,CUST_26548,2024-09-30 00:02:42.882351+00:00,Gas,major,BP,587.57,AUD,Nigeria,mobile,0,False,30-09-2024


In [30]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5989047 entries, 0 to 7483765
Data columns (total 13 columns):
 #   Column               Dtype              
---  ------               -----              
 0   transaction_id       object             
 1   customer_id          object             
 2   timestamp            datetime64[ns, UTC]
 3   merchant_category    object             
 4   merchant_type        object             
 5   merchant             object             
 6   amount               float64            
 7   currency             object             
 8   country              object             
 9   channel              object             
 10  distance_from_home   int64              
 11  weekend_transaction  bool               
 12  date_transact        object             
dtypes: bool(1), datetime64[ns, UTC](1), float64(1), int64(1), object(9)
memory usage: 599.7+ MB


In [31]:
# Convert the timestamp to monthly period
df_transactions['month'] = df_transactions['timestamp'].dt.to_period('M')


/tmp/ipython-input-1536911060.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_transactions['month'] = df_transactions['timestamp'].dt.to_period('M')


In [34]:
# Total spending for each month across all customers
monthly_totals = df_transactions.groupby('month')['amount'].sum()
display(monthly_totals)

,amount
month,
2024-09,2.396796e+08
2024-10,2.343221e+11


In [37]:
# Total spending for each merchant category across all customers
category_totals = df_transactions.groupby('merchant_category')['amount'].sum()
display(category_totals)

,amount
merchant_category,
Education,2.827463e+10
Entertainment,2.172222e+10
Gas,2.808129e+10
Grocery,2.408909e+10
Healthcare,2.797400e+10
Restaurant,2.066437e+10
Retail,3.424239e+10
Travel,4.951381e+10


In [39]:
# Mean spending amount for each merchant category
category_mean = df_transactions.groupby('merchant_category')['amount'].mean()
display(category_mean)

,amount
merchant_category,
Education,37825.286383
Entertainment,28995.131343
Gas,37534.712492
Grocery,32243.291065
Healthcare,37309.759549
Restaurant,27574.849343
Retail,45691.190855
Travel,66148.332302


In [41]:
# Median spending amount for each merchant category
category_median = df_transactions.groupby('merchant_category')['amount'].median()
display(category_median)

,amount
merchant_category,
Education,786.271481
Entertainment,640.225874
Gas,786.670000
Grocery,697.256533
Healthcare,781.200000
Restaurant,605.355000
Retail,1077.330000
Travel,1672.660000


In [43]:
category_stats = df_transactions.groupby('merchant_category')['amount'].agg(['sum', 'mean', 'median'])
display(category_stats)

,sum,mean,median
merchant_category,,,
Education,2.827463e+10,37825.286383,786.271481
Entertainment,2.172222e+10,28995.131343,640.225874
Gas,2.808129e+10,37534.712492,786.670000
Grocery,2.408909e+10,32243.291065,697.256533
Healthcare,2.797400e+10,37309.759549,781.200000
Restaurant,2.066437e+10,27574.849343,605.355000
Retail,3.424239e+10,45691.190855,1077.330000
Travel,4.951381e+10,66148.332302,1672.660000


In [46]:
monthly_category_totals = df_transactions.pivot_table(
    values='amount',
    index='month',
    columns='merchant_category',
    aggfunc='sum'
)

display(monthly_category_totals)

merchant_category,Education,Entertainment,Gas,Grocery,Healthcare,Restaurant,Retail,Travel
month,,,,,,,,
2024-09,2.858407e+07,2.184852e+07,2.511891e+07,2.262619e+07,2.669840e+07,2.505391e+07,3.586817e+07,5.388143e+07
2024-10,2.824604e+10,2.170038e+10,2.805618e+10,2.406647e+10,2.794730e+10,2.063932e+10,3.420653e+10,4.945993e+10


Creating a df for every country and storing in a dict

In [48]:
# Create a dictionary where each key = country name, value = df filtered for that country
country_dfs = {country: df[df['country'] == country]
               for country in df['country'].unique()}

Nested loop to build a JSON-style dictionary

In [49]:
json_dict = {}

for country, df_country in country_dfs.items():
    json_dict[country] = {}

    for category in df_country['merchant_category'].unique():
        df_cat = df_country[df_country['merchant_category'] == category]

        # Example stats: sum, mean, median
        json_dict[country][category] = {
            'total_spent': df_cat['amount'].sum(),
            'mean_spent': df_cat['amount'].mean(),
            'median_spent': df_cat['amount'].median(),
            'transaction_count': len(df_cat)
        }
